This notebook was inspired by this [LlamaIndex example](https://docs.llamaindex.ai/en/stable/examples/cookbooks/llama3_cookbook/).

Making some changes to it with the only intention of trying ideas and learning.

Notice that I am assuming you have the relevant API_KEYs as environmental variables.

To use this notebook you need to agree sharing your profile with meta-llama, [HERE](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct)

You also need to create a HuggingFace token directly on your account.

In [ ]:
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-huggingface

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
import subprocess
import nest_asyncio
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
import subprocess
import os
from transformers import AutoTokenizer
import torch
from llama_index.llms.huggingface import HuggingFaceLLM
nest_asyncio.apply()

## Setup Tokenizer and Stopping ids


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    token=os.environ["hf_token"],
)

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

## Setup Embedding Model and LLM using HuggingFace

In [ ]:
llm = HuggingFaceLLM(
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    model_kwargs={
        "token": os.environ["hf_token"],
        "torch_dtype": torch.bfloat16,
    },
    generate_kwargs={
        "do_sample": True,
        "temperature": 0.6,
        "top_p": 0.9,
    },
    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    tokenizer_kwargs={"token": os.environ["hf_token"]},
    stopping_ids=stopping_ids,
)
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.embed_model = embed_model
Settings.llm = llm

## Defining global variables

In [ ]:
COMPANIES = ["uber", "lyft"]
DATA_DIRS = {}
PERSIST_DIRS = {}
for c in COMPANIES:
    DATA_DIRS[c] = os.path.join(os.environ["DATA_DIR"], f"{c}")
    PERSIST_DIRS[c] = os.path.join(os.environ["PERSIST_DIR"], f"{c}")


## Download data
You can access more files by usinig the next url format

https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/{`COMPANY_NAME`}_10q_{`MONTH`}_{`YEAR`}.pdf'

In [ ]:
for c in COMPANIES:
    if not os.path.exists(DATA_DIRS[c]):
        os.mkdir(DATA_DIRS[c])
        command = f"wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/{c}_2021.pdf' -O '{DATA_DIRS[c]}/{c}_2021.pdf'"        
        subprocess.run(command, shell=True)

# Create index and query engine for each company individually

In [ ]:
query_engine_dict = {}
for c in COMPANIES:
    if not os.path.exists(PERSIST_DIRS[c]):
        print("Creating Index")
        # load the documents and create the index
        documents = SimpleDirectoryReader(DATA_DIRS[c]).load_data()
        index = VectorStoreIndex.from_documents(documents)
        # store it for later
        index.storage_context.persist(persist_dir=PERSIST_DIRS[c])
    else:
        print("Loading Index")
        # load the existing index
        storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIRS[c])
        index = load_index_from_storage(storage_context)
    
    query_engine_dict[c] = index.as_query_engine(similarity_top_k=3)
    

In [ ]:
response = query_engine_dict["uber"].query("What is the revenue of uber in 2021?")
print(response)

In [ ]:
response = query_engine_dict["lyft"].query("What are lyft investments in 2021? Show output as a list with main topics.")
print(response)